<a href="https://colab.research.google.com/github/IA2021UR/trabajo-ia-en-la-comunidad-de-la-rioja-equipo-1/blob/main/limpiezaDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El dataset propuesto contiene aproximadamente 21.000 imágenes para lograr una mayor rapidez en la limpieza de los datos comenzaremos limpiando el dataset de imágenes que no tengan caras, este es el objetivo principal de este cuaderno.

# 1.Descarga del dataset

In [ ]:
!wget https://www.dropbox.com/s/icj8g7cu3krttge/2.zip?dl=0 -O data.zip
!unzip data.zip

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# Leer primer modelo de red convolucional
!wget https://www.dropbox.com/s/uji1ufb009ic3m0/mobilenet_graph.pb?dl=0 -O mobilenet_graph.pb

with tf.io.gfile.GFile('mobilenet_graph.pb','rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

with tf.Graph().as_default() as mobilenet:
    tf.import_graph_def(graph_def,name='')

--2021-05-15 09:59:36--  https://www.dropbox.com/s/uji1ufb009ic3m0/mobilenet_graph.pb?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/uji1ufb009ic3m0/mobilenet_graph.pb [following]
--2021-05-15 09:59:36--  https://www.dropbox.com/s/raw/uji1ufb009ic3m0/mobilenet_graph.pb
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc43fc07b3ce32cb78581e4a8fc5.dl.dropboxusercontent.com/cd/0/inline/BOixmWbGeZ8N0l04fTd6XaQAqtZxHlt8C4RRnW3GcqPOse-Tx1zQaePKkN6N7DV70Oz_LKaIO4Oa1hgK2sTfZxE1OF60-3WfjuvLfGekM3fa1_DzsttBUMS3Ch4W6UpuAJjUv9NMeOjI-UN5vVidRtmY/file# [following]
--2021-05-15 09:59:36--  https://uc43fc07b3ce32cb78581e4a8fc5.dl.dropboxusercontent.com/cd/0/inline/BOixmWbGeZ8N0l04fTd6XaQAqtZxHlt8C4RRnW3GcqPOse-Tx1zQaePKkN6N7DV70Oz

In [ ]:
# Cargar imagen
def load_image(DIR, NAME):
    return cv2.cvtColor(cv2.imread(f'{DIR}/{name}'), cv2.COLOR_BGR2RGB)
def detect_faces(image, score_threshold=0.7):
    global boxes, scores
    (imh, imw) = image.shape[:-1]
    img = np.expand_dims(image,axis=0)
    
    # Inicializar mobilenet
    sess = tf.compat.v1.Session(graph=mobilenet)
    image_tensor = mobilenet.get_tensor_by_name('image_tensor:0')
    boxes = mobilenet.get_tensor_by_name('detection_boxes:0')
    scores = mobilenet.get_tensor_by_name('detection_scores:0')
    
    # Predicción (detección)
    (boxes, scores) = sess.run([boxes, scores], feed_dict={image_tensor:img})
    
    # Reajustar tamaños boxes, scores
    boxes = np.squeeze(boxes,axis=0)
    scores = np.squeeze(scores,axis=0)
    
    # Depurar bounding boxes
    idx = np.where(scores>=score_threshold)[0]
    
    # Crear bounding boxes
    bboxes = []
    for index in idx:
        ymin, xmin, ymax, xmax = boxes[index,:]
        (left, right, top, bottom) = (xmin*imw, xmax*imw, ymin*imh, ymax*imh)
        left, right, top, bottom = int(left), int(right), int(top), int(bottom)
        bboxes.append([left,right,top,bottom])
        
    return bboxes
# Extraer rostros
def extract_faces(image,bboxes,new_size=(160,160)):
    cropped_faces = []
    for box in bboxes:
        left, right, top, bottom = box
        face = image[top:bottom,left:right]
        cropped_faces.append(cv2.resize(face,dsize=new_size))
    return cropped_faces

A partir de estas imágenes obtendremos solo aquellas que tengan cara, para optimizar los resultados estableceremos un tamaño de imagen para todas imágenes de nuestro dataset, esto nos permitirá encontrar aquellas caras más similares en pasos posteriores.

In [ ]:
from os import remove


In [ ]:
print('imagenes originales:'+ str(len(os.listdir('2'))))

imagenes originales:21609


Primero eliminamos las imágenes que producen errores.

In [ ]:
directorio='2'
for name in os.listdir('2'):
      try:
          image= cv2.cvtColor(cv2.imread(f'{directorio}/{name}'), cv2.COLOR_BGR2RGB)   
      except:
          print('Error , la imagen es defectuosa: ' + name )
          remove('2/'+name) 
          
print('fin')         
      
            

fin


Hemos eliminado unas 70 imágenes defectuosas.


Nota: este proceso es muy costoso.

In [ ]:
directorio='2'
num=len(os.listdir('2'))

for name in range (0,num):
          image= cv2.cvtColor(cv2.imread(directorio+'/'+os.listdir('2')[name]), cv2.COLOR_BGR2RGB)
          bboxes = detect_faces(image)
          if len(bboxes)==0:
            print(f'{name}'+ ':' + str(num))
            num=num-1
            remove('2'+'/'+str(os.listdir('2')[name]))     

print('fin')         


In [ ]:
print('imagenes que contienen caras:'+ str(len(os.listdir('2'))))

imagenes que contienen caras:19021


Ahora zipeamos el nuevo dataset

In [ ]:
!zip -r ./3.zip ./2/


Ahora que hemos eliminado unas ** 2500**, es decir hemos limpiado un 10% imágenes que no íbamos a utilizar.
Ahora procedemos a la segunda mejora del dataset, en este caso crearemos el nuevo dataset que estará formado sólamente por caras y permitirá obtener búsquedas más exactas de la cara de una persona.

Ahora descargamos este resultado que guardamos en dropbox por comodidad.

**Nota** He borrado este dataset porque no quedaba espacio en el dropbox,lo he sustituido por este que he obtenido, este será el dataset que utilizaremos en el resto de cuadernos

In [ ]:
!wget https://www.dropbox.com/s/knkdotwfob09y2u/3.zip?dl=0 -O 3.zip
!unzip 3.zip